Many practical problems are similar to optimization problem. This document will show how we solve this type of problems.

Greedy algorithms often provide adequate solution but it is exponential hard. Using dynamic programming often yields good performance for problems.

In the following context, we are going to solve a Knapsack problem.
Distribution:
    I would like to carry several items into my knapsack but there is a maximum weight knapsack that I can carry.

Formalized:
    Each item is represented by <value,weight>    
    $$ max\left(\sum_{i=0}^{n-1} V[i]*I[i].value\right)$$
        where V[i]=1  item[i] is taken, V[i]=0 itme[i] isn't taken
    
    subject to 
    
    $$ \left(\sum_{i=0}^{n-1} V[i]*I[i].value \right)<=w$$
    



Bruce Force Algorithm
    1. Generate power set
    2. Remove all of the sets whose total units exceeds the allowed weight
    3. From the remaining sets, choose one whose value is the largest 
    
Cons: exponential complexity $2^n$

Greedy Algorithm
While knapack is not full, we put "best" available itme in knapack.
    1. Define the metric. It can be the value, cost, weight,..
    2. Sort by the metric.
    3. Put items in the knapack until the knapack is full.    
    
Cons: a bit bitter but it's still exponential complexity n(log n)

In [30]:
import random

In [27]:
class Food(object):
    def __init__(self, n, v, w):
        self.name = n
        self.value = v
        self.calories = w
    def getValue(self):
        return self.value
    def getCost(self):
        return self.calories
    def density(self):
        return self.getValue()/self.getCost()
    def __str__(self):
        return self.name + ': <' + str(self.value)\
                 + ', ' + str(self.calories) + '>'

def buildMenu(names, values, calories):
    """names, values, calories lists of same length.
       name a list of strings
       values and calories lists of numbers
       returns list of Foods"""
    menu = []
    for i in range(len(values)):
        menu.append(Food(names[i], values[i],
                          calories[i]))
    return menu

def buildLargeMenu(numItems, maxVal, maxCost):
    items = []
    for i in range(numItems):
        items.append(Food(str(i),
                          random.randint(1, maxVal),
                          random.randint(1, maxCost)))
    return items

In [21]:
def greedy(items, maxCost, keyFunction):
    """Assumes items a list, maxCost >= 0,
         keyFunction maps elements of items to numbers"""
    itemsCopy = sorted(items, key = keyFunction,
                       reverse = True)
    result = []
    totalValue, totalCost = 0.0, 0.0
    for i in range(len(itemsCopy)):
        if (totalCost+itemsCopy[i].getCost()) <= maxCost:
            result.append(itemsCopy[i])
            totalCost += itemsCopy[i].getCost()
            totalValue += itemsCopy[i].getValue()
    return (result, totalValue)

In [22]:
def testGreedy(items, constraint, keyFunction):
    taken, val = greedy(items, constraint, keyFunction)
    print('Total value of items taken =', val)
    for item in taken:
        print('   ', item)

def testGreedys(foods, maxUnits):
    print('Use greedy by value to allocate', maxUnits,
          'calories')
    testGreedy(foods, maxUnits, Food.getValue)
    print('\nUse greedy by cost to allocate', maxUnits,
          'calories')
    testGreedy(foods, maxUnits,
               lambda x: 1/Food.getCost(x))
    print('\nUse greedy by density to allocate', maxUnits,
          'calories')
    testGreedy(foods, maxUnits, Food.density)

In [23]:
names = ['wine', 'beer', 'pizza', 'burger', 'fries',
         'cola', 'apple', 'donut', 'cake']
values = [89,90,95,100,90,79,50,10]
calories = [123,154,258,354,365,150,95,195]
foods = buildMenu(names, values, calories)
testGreedys(foods, 1000)

Use greedy by value to allocate 1000 calories
Total value of items taken = 424.0
    burger: <100, 354>
    pizza: <95, 258>
    beer: <90, 154>
    wine: <89, 123>
    apple: <50, 95>

Use greedy by cost to allocate 1000 calories
Total value of items taken = 413.0
    apple: <50, 95>
    wine: <89, 123>
    cola: <79, 150>
    beer: <90, 154>
    donut: <10, 195>
    pizza: <95, 258>

Use greedy by density to allocate 1000 calories
Total value of items taken = 413.0
    wine: <89, 123>
    beer: <90, 154>
    cola: <79, 150>
    apple: <50, 95>
    pizza: <95, 258>
    donut: <10, 195>


However, Greedy algorithm can't guarantee that we can get the optimal solutions

Search Tree
    Each node=<taken, left, totalValue, remaining calories>
    1. The tree is built top down starting the root.
    2. The first element is selected from the still to be considered items
        If it's chosen, a node is constructed that reflects the consequence of choosing to take that item.
        If it is not chosen, a node is constructed that reflects the consequence of not taking that item.
    3. Applied recursively until there is no leaf children
    4. Choos a node with the highest value

However, adding memo as a new argument can improve our algorithm

def fastMaxVal(toConsider, avail, memo = {}):
where momo is a tuple
    (items left to be considered, available weight)

In [34]:
def maxVal(toConsider, avail):
    """Assumes toConsider a list of items, avail a weight
       Returns a tuple of the total weight of a solution to the
         0/1 knapsack problem and the items of that solution"""
    if toConsider == [] or avail == 0:
        result = (0, ())
    elif toConsider[0].getCost() > avail:
        #Explore right branch only
        result = maxVal(toConsider[1:], avail)
    else:
        nextItem = toConsider[0]
        #Explore left branch
        withVal, withToTake = maxVal(toConsider[1:],
                                     avail - nextItem.getCost())
        withVal += nextItem.getValue()
        #Explore right branch
        withoutVal, withoutToTake = maxVal(toConsider[1:], avail)
        #Choose better branch
        if withVal > withoutVal:
            result = (withVal, withToTake + (nextItem,))
        else:
            result = (withoutVal, withoutToTake)
    return result



def fastMaxVal(toConsider, avail, memo = {}):
    """Assumes toConsider a list of subjects, avail a weight
         memo supplied by recursive calls
       Returns a tuple of the total value of a solution to the
         0/1 knapsack problem and the subjects of that solution"""
    if (len(toConsider), avail) in memo:
        result = memo[(len(toConsider), avail)]
    elif toConsider == [] or avail == 0:
        result = (0, ())
    elif toConsider[0].getCost() > avail:
        #Explore right branch only
        result = fastMaxVal(toConsider[1:], avail, memo)
    else:
        nextItem = toConsider[0]
        #Explore left branch
        withVal, withToTake =\
                 fastMaxVal(toConsider[1:],
                            avail - nextItem.getCost(), memo)
        withVal += nextItem.getValue()
        #Explore right branch
        withoutVal, withoutToTake = fastMaxVal(toConsider[1:],
                                                avail, memo)
        #Choose better branch
        if withVal > withoutVal:
            result = (withVal, withToTake + (nextItem,))
        else:
            result = (withoutVal, withoutToTake)
    memo[(len(toConsider), avail)] = result
    return result

def testMaxVal(foods, maxUnits, algorithm, printItems = True):
    print('Menu contains', len(foods), 'items')
    print('Use search tree to allocate', maxUnits,
          'calories')
    val, taken = algorithm(foods, maxUnits)
    if printItems:
        print('Total value of items taken =', val)
        for item in taken:
            print('   ', item)

In [ ]:
for numItems in (5, 10, 15, 20, 25, 30, 35, 40, 45, 50):
    items = buildLargeMenu(numItems, 90, 250)
    testMaxVal(items, 750, maxVal, False)

Menu contains 5 items
Use search tree to allocate 750 calories
Menu contains 10 items
Use search tree to allocate 750 calories
Menu contains 15 items
Use search tree to allocate 750 calories
Menu contains 20 items
Use search tree to allocate 750 calories
Menu contains 25 items
Use search tree to allocate 750 calories
Menu contains 30 items
Use search tree to allocate 750 calories
Menu contains 35 items
Use search tree to allocate 750 calories
Menu contains 40 items
Use search tree to allocate 750 calories
